In [1]:
# header files
import numpy as np
import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
# load cifar-10 data
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(root="./data", train=True, transform=transforms.Compose([
                                            transforms.RandomHorizontalFlip(),
                                            transforms.RandomCrop(32, 4),
                                            transforms.ToTensor(),
                                            normalize,
                                            ]), download = True)
                                           , batch_size=128, shuffle=True, num_workers=4, pin_memory=True)

val_loader = torch.utils.data.DataLoader(datasets.CIFAR10(root="./data", train=False, transform=transforms.Compose([
                                            transforms.ToTensor(),
                                            normalize,
                                            ])), batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

Files already downloaded and verified


In [3]:
# analyse data
data = iter(train_loader)
image = data.next()
print(image[0][0])

tensor([[[ 0.5364,  0.5364,  0.5364,  ...,  0.6392,  0.6734,  0.6734],
         [ 0.5707,  0.5536,  0.5536,  ...,  0.6392,  0.6734,  0.6906],
         [ 0.6221,  0.6049,  0.6049,  ...,  0.6392,  0.6563,  0.7248],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[ 1.9209,  1.9034,  1.9209,  ...,  1.9209,  1.9559,  1.9734],
         [ 1.9384,  1.9209,  1.9209,  ...,  1.9734,  1.9734,  1.9909],
         [ 1.9559,  1.9209,  1.9209,  ...,  2.0084,  1.9909,  2.0259],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[ 2.6400,  2.6400,  2.6400,  ...,  2.6400,  2.6226,  2.6051],
         [ 2.6400,  2.6226,  2.6226,  ...,  2

In [4]:
# define loss for network
criterion = nn.CrossEntropyLoss()

In [5]:
# define VGG16 network
class VGG16(nn.Module):
    
    # init method
    def __init__(self, num_classes = 10):
        super(VGG16, self).__init__()
        
        self.features = nn.Sequential(
            
            # first cnn block
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # second cnn block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # third cnn block
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # fourth cnn block
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # fifth cnn block
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        #self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, num_classes)
        )
        
    
    # forward step
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [6]:
model = VGG16()

In [7]:
# optimizer to be used
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=5e-4)

In [8]:
# train and validate
for epoch in range(0, 10):
    
    # train
    model.train()
    training_loss = 0.0
    for i, (input, target) in enumerate(train_loader):
        
        output = model(input)
        loss = criterion(output, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        training_loss = training_loss + (loss.item() * 128)
    training_loss = training_loss / len(train_loader.sampler)
    
    # validate
    model.eval()
    valid_loss = 0.0
    for i, (input, target) in enumerate(val_loader):
        
        with torch.no_grad():
            output = model(input)
            loss = criterion(output, target)
            
        valid_loss = valid_loss + (loss.item() * 128)
    valid_loss = valid_loss / len(val_loader.sampler)
    
    print("Training loss: " + str(training_loss) + "    Validation loss: " + str(valid_loss))
    print()

Training loss: 2.30617158325    Validation loss: 2.32938701782
()
Training loss: 2.30601085571    Validation loss: 2.32924762573
()
Training loss: 2.30589680115    Validation loss: 2.32894863281
()
Training loss: 2.30568252441    Validation loss: 2.32919759216
()
Training loss: 2.30585125366    Validation loss: 2.32944734497
()
Training loss: 2.3056326178    Validation loss: 2.3289
()
Training loss: 2.30576660706    Validation loss: 2.32881203613
()
Training loss: 2.30568395142    Validation loss: 2.32914135437
()
Training loss: 2.30558817627    Validation loss: 2.32905328979
()
Training loss: 2.30564895264    Validation loss: 2.32954781494
()
